## Pre-Processing the Files

- Considers two files as input namely data file (matrixFile) and meta data file (metaDataFile)
- Merged them based on cellCodeBar value
- Identify the columns where all values are zero
- Delete the columns with zero values
- Create Dummy Variables for Cell Type
- Saves the resultant data frame to CSV
- The output (outputFile) is used as input for XGBoost Classifier

In [ ]:
import pandas as pd

In [2]:
matrixFile = 'Matrix.csv'
metaDataFile = 'metadata.csv'
outputFile = 'OutputFile-Dummy.csv'

In [ ]:
# Load the meta data file
df_metaData = pd.read_csv(metaDataFile)
df_metaData.head()

In [ ]:
#Depending on the data, some cols which are not required are dropped
df_metaData.drop(['orig.ident'], axis = 1,inplace=True)
df_metaData.drop(['nCount_RNA'], axis = 1,inplace=True)
df_metaData.drop(['nFeature_RNA'], axis = 1,inplace=True)
df_metaData.rename(columns = {'Unnamed: 0':'cellBarCode'}, inplace = True)
label = "cellBarCode"
df_metaData.head()

In [ ]:
# Load the matrix/data file
df_Matrix = pd.read_csv(MatrixFile)
df_Matrix.head()

In [ ]:
# Taking transpose
df_Matrix = df_Matrix.T

In [ ]:
# Assigning the proper header 
new_header = df_Matrix.iloc[0]
df_Matrix = df_Matrix[1:]
df_Matrix.columns = new_header
df_Matrix.head()

In [ ]:
# Resetting the index
df_Matrix = df_Matrix.reset_index(level=0)
df_Matrix.head()

In [ ]:
df_Matrix.rename(columns = {'gene':'index_1'}, inplace = True)
df_Matrix.rename(columns = {'index':label+"_Orig"}, inplace = True)
df_Matrix.head()

In [ ]:
df = pd.merge(df_Matrix, df_metaData, on=label, how='inner')

In [ ]:
df.drop(['orig.ident','nCount_RNA','nFeature_RNA'], axis = 1,inplace=True)

## Pre-Processing the Data Frame

In [ ]:
if 'SampleID' in df.columns:
    df.drop("SampleID", axis = 1, inplace=True)

In [ ]:
#Removes columns with zero values
df = df.loc[:, df.any()]

In [ ]:
#Getting the label counts
label_counts = df["celltype"].value_counts()
label_counts

In [ ]:
%%time
df = pd.concat([df,pd.get_dummies(df['celltype'])],axis=1)
df.drop('celltype', axis=1, inplace=True)
df.head()

In [ ]:
#checking for newly created cols
df.shape

In [ ]:
%%time
df.to_csv(outputFile, encoding='utf-8', index=False)